---
# **LA LOI DU CROUS :**
## *Étude des déterminations du prix des résidences universitaires Crous et de leurs répartitions dans les départements*
---

# Introduction:

350 000 étudiants habitent dans une résidence des centres régionaux des œuvres universitaires et scolaires (Crous), soit seulement 10% des 2 million d'étudiants vivant hors de leur domiciles, selon un  rapport sur le logement et la précarité des étudiants, des apprentis et des jeunes actifs de l'Assemblée nationale datant de janvier 2022 [1]. Selon ce même rapport, le logement est le premier poste de dépense des étudiants soit 60% de leurs revenus. Il est donc primordiale de donner accès à un logement abordable aux étudiants qui sont de plus en plus précaires de manières équitables selon les départements où ils se trouvent. 

Partant de ces constats, nous nous sommes posés plusieurs questions : le loyer est-il le même pour un logement situé dans un marché immobilier en tension comme Paris que dans un marché qu'il l'est moins comme Dijon ? Ou au contraire, est-ce que les prix sont globalement uniformes ? De même, est-ce que la concentration importante d'étudiants scolarisés à proximité de la résidence influe le prix vers la hausse? 

De ce fait, nous chercherons donc à répondre à la problématique suivante : 

***Les loyers des résidences des Crous suivent-ils la loi de l'offre et de la demande ?***

Ainsi, nous proposons d'étudier dans ce rapport, d'une part la répartition des 795 résidences des Crous sur le territoire; et d'autre part, les déterminants du prix du loyer de celles-ci pour pouvoir montrer d'éventuelles variations entre les départements. Pour expliquer le loyer nous utiliserons des méthodes de régressions linéaires classiques à partir d'un modèle d'offre et de demande économique élèmentaire. Dans celui-ci, la résidence offre des biens de surfaces variables q à un prix p en mètre carré, et fait face à une concurrence représenté par le montant des loyers dans le parc privé. La demande sera représentée par l'effectifs d'étudiants scolarisés dans une école se situant dans un rayon de 20km de la résidence. Nous faisons l'hypothèse que les résidences proposent à peu près les mêmes caractéristiques entre elles et que les étudiants sont des price-takers. 

**PLAN:**

1. Dans une première partie, nous réccupérons une base de données avec les loyers (maximum, minimum et moyen), les surfaces (idem), les coordonnées géographiques, les loyers privés dans la ville de la résidence et l'effectif des étudiants dans un périmètre de 20 kilomètres. 

2. Dans une deuxième partie, nous analyserons d'une part la répartition des résidences en France et, d'autre part, nous réaliserons des statistiques descriptives sur les différentes variables de notre base de données. 

3. Dernièrement, nous utiliserons des modèles de régressions linéaires pour répondre à notre problématique. 


## Les données utilisées 

Nous nous servirons de plusieures base de données : 

1. **Ensemble des logements proposés aux étudiants par le réseau des CROUS :** 

Données mises à disposition par le Centre National des Œuvres Universitaires et Scolaires (CNOUS). Ils ont été publiés en 2017 et sont mises à jour assez régulièrement. Le problème principale de cette base de données est la récupération des loyers et des surfaces qui sont stockés dans une colonne nommée "infos" (s'il y en a). De plus, chaque colonne a été remplies par les résidences. La qualité et la structure des informations sont donc très variables. Nous avons donc mis en place des techniques de traitements textuels ou de natural language processing (NPL).

https://www.data.gouv.fr/fr/datasets/ensemble-des-logements-proposes-aux-etudiants-par-le-reseau-des-crous/

2. **"Carte des loyers" - Indicateurs de loyers d'annonce par commune en 2018**

Données obtenues par une collaboration entre la Direction Générale de l’Aménagement, du Logement et de la Nature (DGALN), une équipe de recherche en économie d’Agrosup Dijon, de l’Institut national de la recherche en agronomique (INRAE), de SeLoger, de leboncoin et de PAP. Ces équipes ont créé différent indicateurs par communes à partir de 9 million d'annonces. Les données datent de 2018 et nous nous servirons uniquement de données pour les appartements.

https://www.data.gouv.fr/fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2018/


3. **Statistiques sur les effectifs d'étudiants inscrits par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur (avec doubles inscriptions CPGE)**

"Ce jeu de données propose un ensemble de statistiques sur les effectifs d'étudiants inscrits de 2006-07 à 2021-22 par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur français : universités, universités de technologie, grands établissements, COMUE, écoles normales supérieures, écoles centrales, INSA, autres écoles d'ingénieurs..."

https://www.data.gouv.fr/fr/datasets/statistiques-sur-les-effectifs-detudiants-inscrits-par-etablissement-public-sous-tutelle-du-ministere-en-charge-de-lenseignement-superieur-avec-doubles-inscriptions-cpge/

4. **Contours Communes France Administrative "Format Admin-Express" avec arrondissements**

Jeu de données de l'IGN regroupant les contours communales avec arrondissements.

https://www.data.gouv.fr/fr/datasets/contours-communes-france-administrative-format-admin-express-avec-arrondissements/

5. **Contours des départements français issus d'OpenStreetMap**

Jeu de données d'OpenStreetMap regroupant les contours géographiques des départements. 

https://www.data.gouv.fr/fr/datasets/contours-des-departements-francais-issus-d-openstreetmap/





# Librairies nécessaires

In [1]:
%%capture 
#hide output 
!pip install geoplot
!pip install contextily
!pip install altair

In [2]:
import pandas as pd 
import geopandas as gpd 
import nltk
nltk.download('punkt')
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import altair as alt 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


# Partie I : Obtention du dataframe 

## I.1. Récupération des loyers, des surfaces et de la localisation des résidences

In [3]:
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr//explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", 
                 sep =';')


In [4]:
df = df[["title", "infos", "address" ,"geocalisation", "regions"]]

In [5]:
from functions_for_data_cleaning import *

df = get_loyer(df)
df = get_surface(df)
df = get_localisation(df) 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## I.2. Récupération des loyers privés des communes (avec la géolocalisation des communes)

In [6]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")

,id_zone,INSEE,LIBGEO,DEP,REG,EPCI,TYPPRED,loypredm2,lwr.IPm2,upr.IPm2,R2adj,NBobs_maille,NBobs_commune
0,2362,1001,L'Abergement-Clémenciat,1,84,200069193,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,2
1,2010,1002,L'Abergement-de-Varey,1,84,240100883,maille,"8,63555202","6,72107751","11,09535764","0,719598662",861,0
2,1456,1004,Ambérieu-en-Bugey,1,84,240100883,commune,"10,07450708","7,865307059","12,90422511","0,670160882",2156,2156
3,2362,1005,Ambérieux-en-Dombes,1,84,200042497,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,81
4,2055,1006,Ambléon,1,84,200040350,maille,"8,96695486","7,101593346","11,3222872","0,752823051",842,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35436,1802,97420,Sainte-Suzanne,974,4,249740119,commune,"10,67356968","8,791530189","12,95850521","0,788143839",847,411
35437,1802,97421,Salazie,974,4,249740093,maille,"10,38745938","8,545191709","12,62690364","0,788143839",847,11
35438,1608,97422,Le Tampon,974,4,249740085,commune,"10,81548488","8,696332802","13,45103918","0,81541405",3237,3237
35439,1607,97423,Les Trois-Bassins,974,4,249740101,maille,"12,34315971","8,597802405","17,72006199","0,460731322",1669,90


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [7]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [8]:
df_private.rename(columns = {'INSEE':'codgeo',"upr.IPm2" : "Loyer_prive_m2" }, inplace = True)

/tmp/ipykernel_2033/3051210400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_private.rename(columns = {'INSEE':'codgeo',"upr.IPm2" : "Loyer_prive_m2" }, inplace = True)


In [9]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base).
"""
df_private["codgeo"] = df_private["codgeo"].apply(lambda x : "0" + x if len(x) == 4 else x)


/tmp/ipykernel_2033/1230424880.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_private["codgeo"] = df_private["codgeo"].apply(lambda x : "0" + x if len(x) == 4 else x)


In [10]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes.(prends du temps)

In [11]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [13]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## I.3. Jointures spatiales

In [14]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_2033/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [15]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure

,title,infos,address,geocalisation,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer,Surface,Max Surface,Min Surface,Mean Surface,Longitude,Latitude,geometry,LIBGEO,Loyer_prive_m2
0,Résidence du Nain d'Alsace,Localisation\r\n A proximité du centre ville \...,Rue Quentin Barre 02100 Saint-Quentin,"49.8493385315,3.3014810085",Hauts-de-France,"[266.23, 312.87]",312.87,266.23,289.550000,"[14.5, 19.5]",19.5,14.5,17.000000,3.301481,49.849339,POINT (3.30148 49.84934),Saint-Quentin,"11,99298283"
1,Résidence du Thil,"Localisation\r\n À Amiens sud, à proximité du ...",Avenue des Facultés 80025 Amiens Cédex 1,"49.871897,2.26004",Hauts-de-France,"[256.74, 349.46, 298.66]",349.46,256.74,301.620000,[],NaN,NaN,NaN,2.260040,49.871897,POINT (2.26004 49.87190),Salouël,"14,62876701"
2,Résidence du Beffroi,"Localisation\r\n En centre ville, à proximité ...",29 rue au Lin 80000 Amiens,"49.8958244324,2.2954077721",Hauts-de-France,"[301.18, 410.29, 524.61]",524.61,301.18,412.026667,"[19.0, 35.0, 50.0]",50.0,19.0,34.666667,2.295408,49.895824,POINT (2.29541 49.89582),Amiens,"14,91402079"
3,Résidence Charles de Gaulle,"Localisation\r\n Face à la Citadelle, à proxim...",NaN,"49.9027786255,2.3004710674",Hauts-de-France,"[342.61, 455.83, 362.41, 488.0, 536.22, 590.05...",607.35,342.61,483.210000,"[16.9, 26.35, 18.5, 28.65, 30.5, 53.4]",53.4,16.9,29.050000,2.300471,49.902779,POINT (2.30047 49.90278),Amiens,"14,91402079"
4,Résidence Friant,"Localisation\r\n À l'ouest d'Amiens, entre le ...",54/58 square Friant 80000 Amiens,"49.8907203674,2.2777562141",Hauts-de-France,"[329.18, 407.13]",407.13,329.18,368.155000,"[13.0, 18.0]",18.0,13.0,15.500000,2.277756,49.890720,POINT (2.27776 49.89072),Amiens,"14,91402079"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,Résidence Cap Avenue (secteur Rouen ),Description\nStudios ouvrant le droit à l'APL ...,"118 Avenue de Caen, 76100 Rouen","49.429,1.072",Normandie,[360.0],360.00,360.00,360.000000,[16.0],16.0,16.0,16.000000,1.072000,49.429000,POINT (1.07200 49.42900),Rouen,"14,85000357"
791,Résidence Les Alternants,Plus d'images sur Pinterest\n \nCette résidenc...,Route du Rhin 67400 Illkirch,"48.5205001831,7.7310600281",Grand Est,[],NaN,NaN,NaN,[18.0],18.0,18.0,18.000000,7.731060,48.520500,POINT (7.73106 48.52050),Illkirch-Graffenstaden,"15,35008998"
792,Résidence Les Hauts de L’Illberg,Plus d'images sur Pinterest\n \n\n\nLoyer : 38...,13 rue des Frères Lumière 68350 Brunstatt,"47.7276000977,7.3100199699",Grand Est,[381.0],381.00,381.00,381.000000,[18.0],18.0,18.0,18.000000,7.310020,47.727600,POINT (7.31002 47.72760),Brunstatt-Didenheim,"15,02208494"
793,Résidence Les Cattleyas,Plus d'images sur Pinterest\n \n\n\nLoyer : 41...,"2 rue du Vieil Armand, 67100 Strasbourg","48.5666007996,7.7768602371",Grand Est,[414.0],414.00,414.00,414.000000,[18.0],18.0,18.0,18.000000,7.776860,48.566601,POINT (7.77686 48.56660),Strasbourg,"16,88901671"


## I.4 Ajout des départements 

In [16]:
gdf_dep = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/eb36371a-761d-44a8-93ec-3d728bec17ce")
gdf_dep.rename(columns = {'nom':'departement'}, inplace = True)
gdf_dep = gdf_dep[["departement", "geometry"]]

,departement,geometry
0,La Réunion,"MULTIPOLYGON (((55.21643 -21.03904, 55.21652 -..."
1,Aude,"POLYGON ((1.68872 43.27368, 1.69001 43.27423, ..."
2,Haute-Loire,"POLYGON ((3.08206 45.28988, 3.08209 45.29031, ..."
3,Bouches-du-Rhône,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4..."
4,Lot-et-Garonne,"POLYGON ((-0.14058 44.22648, -0.12931 44.23218..."
...,...,...
97,Morbihan,"MULTIPOLYGON (((-3.73508 48.11140, -3.73507 48..."
98,Doubs,"POLYGON ((5.69876 47.26464, 5.69877 47.26481, ..."
99,Jura,"MULTIPOLYGON (((5.25202 46.94451, 5.25208 46.9..."
100,Ardèche,"POLYGON ((3.86110 44.71118, 3.86110 44.71151, ..."


In [17]:
df = gpd.sjoin(df, gdf_dep, how='left', op='within')
del df['index_right'] #on supprime l'index right pour la prochaine jointure

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_2033/3363061827.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_dep, how='left', op='within')


## I.5. Association de résidences à une école 

Import de notre base de données des écoles.

In [24]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/0c713161-26fb-415e-ac1d-8769125f338d", sep = ";")

,attention,annee_universitaire,etablissement_type,etablissement_typologie,etablissement_id_paysage,etablissement_lib,etablissement_id_wikidata,etablissement_id_ror,etablissement_id_uai,etablissement_localisation,...,etablissement_id_departement,etablissement_departement,etablissement_id_uucr,etablissement_uucr,etablissement_code_commune,etablissement_commune,etablissement_id_paysage_actuel,etablissement_actuel_lib,rentree,annee
0,Sans double compte des établissements-composan...,2006-07,Université,Université pluridisciplinaire avec santé,06SE7,Université de Bretagne Occidentale,Q1857334,01b8h3982,0290346U,Bretagne>Rennes>Finistère>Brest,...,D029,Finistère,UU29701,Brest,29019,Brest,06SE7,Université de Bretagne Occidentale,2006,2007
1,Sans double compte des établissements-composan...,2006-07,Université,Université tertiaire - droit et économie,542Id,Université Toulouse Capitole,Q590201,0443n9e75,0311382J,Occitanie>Toulouse>Haute-Garonne>Toulouse,...,D031,Haute-Garonne,UU00758,Toulouse,31555,Toulouse,542Id,Université Toulouse Capitole,2006,2007
2,Sans double compte des établissements-composan...,2006-07,Autre établissement,NaN,5y7lF,Institut universitaire de formation des maître...,NaN,NaN,9730224F,Guyane>Cayenne,...,D973,Guyane,UU9C601,Cayenne,97302,Cayenne,hy4EW,Université de Guyane,2006,2007
3,Sans double compte des établissements-composan...,2006-07,Université,Université pluridisciplinaire avec santé,7hB8r,Université de Nantes,Q259388,03gnr7b55,0440984F,Pays de la Loire>Nantes>Loire-Atlantique>Nantes,...,D044,Loire-Atlantique,UU44701,Nantes,44109,Nantes,qUCQp,Nantes Université,2006,2007
4,Sans double compte des établissements-composan...,2006-07,École,NaN,8618D,École normale supérieure PSL,Q83259,05a0dhs15,0753455Y,Île-de-France>Paris>Paris 5e,...,D075,Paris,UU00851,Paris,75105,Paris 5e,8618D,École normale supérieure PSL,2006,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,Sans double compte des établissements-composan...,2021-22,École,NaN,tdEpy,École nationale supérieure de l'électronique e...,Q2707341,03qeacd72,0951376E,Île-de-France>Versailles>Val-d'Oise>Paris>Cergy,...,D095,Val-d'Oise,UU00851,Paris,95127,Cergy,tdEpy,École nationale supérieure de l'électronique e...,2021,2022
2284,Sans double compte des établissements-composan...,2021-22,Université,Université pluridisciplinaire avec santé,vb71K,Université Paris-Est Créteil,Q980688,05ggc9x40,0941111X,Île-de-France>Créteil>Val-de-Marne>Paris>Créteil,...,D094,Val-de-Marne,UU00851,Paris,94028,Créteil,vb71K,Université Paris-Est Créteil,2021,2022
2285,Sans double compte des établissements-composan...,2021-22,Université,Université pluridisciplinaire avec santé,xJdyB,Aix-Marseille Université,Q2302586,035xkbk20,0134009M,Provence-Alpes-Côte d'Azur>Aix-Marseille>Bouch...,...,D013,Bouches-du-Rhône,UU00759,Marseille-Aix-en-Provence,13207,Marseille 7e,xJdyB,Aix-Marseille Université,2021,2022
2286,Sans double compte des établissements-composan...,2021-22,École,NaN,zHvr4,École nationale supérieure des arts et techniq...,Q3578242,NaN,0693735K,Auvergne-Rhône-Alpes>Lyon>Rhône>Lyon>Lyon 5e,...,D069,Rhône,UU00760,Lyon,69385,Lyon 5e,zHvr4,École nationale supérieure des arts et techniq...,2021,2022


In [19]:
df_schools = df_schools[df_schools["annee_universitaire"] == "2021-22" ] #on ne prend que l'année 2021-22
df_schools = df_schools[["etablissement_lib", "effectif_sans_cpge", "etablissement_code_commune", "etablissement_commune"]]
df_schools.rename(columns = {'etablissement_code_commune':'codgeo'}, inplace = True) #On renomme la colonne pour la jointure

On enlève les valeurs manquantes (il manque des coordonnées pour certains établissments).

In [20]:
index_with_nan = df_schools.index[df_schools.isnull().any(axis=1)]
df_schools.drop(index_with_nan,0, inplace=True) 
df_schools.set_index('codgeo', inplace = True) #on recommence l'index

/tmp/ipykernel_2033/2476089590.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_schools.drop(index_with_nan,0, inplace=True)


Jointure avec les coordonnées géographiques des communes pour pouvoir trouver dans un rayon de 20km les écoles autour des résidences.

In [21]:
df_schools = df_schools.join(gdf_communes_boundaries)
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.crs = "epsg:4326"

On applique la fonction get_nb_student disponible dans le fichier function_for_data_cleaning.py qui permet d'obtenir le nombre d'étudiants dans un rayon de 20km autour de nos résidences.

In [22]:
df.crs = "epsg:4326" #on définit la projection de notre dataframe
df = get_nb_student(df, df_schools, 20)

## I.6. Obtention de la base de données finale

In [23]:
df 

,title,infos,address,geocalisation,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer,Surface,...,Min Surface,Mean Surface,Longitude,Latitude,geometry,LIBGEO,Loyer_prive_m2,departement,Schools,Nbstudents
0,Résidence du Nain d'Alsace,Localisation\r\n A proximité du centre ville \...,Rue Quentin Barre 02100 Saint-Quentin,"49.8493385315,3.3014810085",Hauts-de-France,"[266.23, 312.87]",312.87,266.23,289.550000,"[14.5, 19.5]",...,14.5,17.000000,3.301481,49.849339,POINT (3.30148 49.84934),Saint-Quentin,"11,99298283",Aisne,[],0
1,Résidence du Thil,"Localisation\r\n À Amiens sud, à proximité du ...",Avenue des Facultés 80025 Amiens Cédex 1,"49.871897,2.26004",Hauts-de-France,"[256.74, 349.46, 298.66]",349.46,256.74,301.620000,[],...,NaN,NaN,2.260040,49.871897,POINT (2.26004 49.87190),Salouël,"14,62876701",Somme,[Université de Picardie Jules-Verne],32266
2,Résidence du Beffroi,"Localisation\r\n En centre ville, à proximité ...",29 rue au Lin 80000 Amiens,"49.8958244324,2.2954077721",Hauts-de-France,"[301.18, 410.29, 524.61]",524.61,301.18,412.026667,"[19.0, 35.0, 50.0]",...,19.0,34.666667,2.295408,49.895824,POINT (2.29541 49.89582),Amiens,"14,91402079",Somme,[Université de Picardie Jules-Verne],32266
3,Résidence Charles de Gaulle,"Localisation\r\n Face à la Citadelle, à proxim...",NaN,"49.9027786255,2.3004710674",Hauts-de-France,"[342.61, 455.83, 362.41, 488.0, 536.22, 590.05...",607.35,342.61,483.210000,"[16.9, 26.35, 18.5, 28.65, 30.5, 53.4]",...,16.9,29.050000,2.300471,49.902779,POINT (2.30047 49.90278),Amiens,"14,91402079",Somme,[Université de Picardie Jules-Verne],32266
4,Résidence Friant,"Localisation\r\n À l'ouest d'Amiens, entre le ...",54/58 square Friant 80000 Amiens,"49.8907203674,2.2777562141",Hauts-de-France,"[329.18, 407.13]",407.13,329.18,368.155000,"[13.0, 18.0]",...,13.0,15.500000,2.277756,49.890720,POINT (2.27776 49.89072),Amiens,"14,91402079",Somme,[Université de Picardie Jules-Verne],32266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,Résidence Cap Avenue (secteur Rouen ),Description\nStudios ouvrant le droit à l'APL ...,"118 Avenue de Caen, 76100 Rouen","49.429,1.072",Normandie,[360.0],360.00,360.00,360.000000,[16.0],...,16.0,16.000000,1.072000,49.429000,POINT (1.07200 49.42900),Rouen,"14,85000357",Seine-Maritime,"[Université de Rouen Normandie, Institut natio...",36442
791,Résidence Les Alternants,Plus d'images sur Pinterest\n \nCette résidenc...,Route du Rhin 67400 Illkirch,"48.5205001831,7.7310600281",Grand Est,[],NaN,NaN,NaN,[18.0],...,18.0,18.000000,7.731060,48.520500,POINT (7.73106 48.52050),Illkirch-Graffenstaden,"15,35008998",Bas-Rhin,[],0
792,Résidence Les Hauts de L’Illberg,Plus d'images sur Pinterest\n \n\n\nLoyer : 38...,13 rue des Frères Lumière 68350 Brunstatt,"47.7276000977,7.3100199699",Grand Est,[381.0],381.00,381.00,381.000000,[18.0],...,18.0,18.000000,7.310020,47.727600,POINT (7.31002 47.72760),Brunstatt-Didenheim,"15,02208494",Haut-Rhin,[Université de Haute-Alsace],10029
793,Résidence Les Cattleyas,Plus d'images sur Pinterest\n \n\n\nLoyer : 41...,"2 rue du Vieil Armand, 67100 Strasbourg","48.5666007996,7.7768602371",Grand Est,[414.0],414.00,414.00,414.000000,[18.0],...,18.0,18.000000,7.776860,48.566601,POINT (7.77686 48.56660),Strasbourg,"16,88901671",Bas-Rhin,"[Université de Strasbourg, Institut national d...",56593
